In [1]:
import timm
from torchsummary import summary
import torch
from torch import nn
import torch.nn.functional as F
from importlib import reload
import cv2
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import albumentations
from albumentations import augmentations
from tqdm import tqdm
from tqdm import tqdm_notebook
from sklearn import model_selection

In [2]:
import albumentations.pytorch

df = pd.read_csv("tensor_sample.csv")

real = df[df["label"] == 0].sample(n=3000)
fakes = df[df["label"] == 1].sample(n=3000)

data = pd.concat([fakes, real])
data = data.values

In [3]:
from segmentation.merged_net import SRM_Classifer
model = SRM_Classifer().to('cuda')
model = nn.DataParallel(model)

In [12]:
import image_ensemble
reload(image_ensemble)
from image_ensemble import ensemble

root_dir = 'Image_Manipulation_Dataset/CASIA_2.0'
out_dir = 'Image_Manipulation_Dataset/CASIA_2.0/1792_tensors'
for row in tqdm_notebook(data):
    image_patch, mask_patch, label, fold, ela = row

    fname = image_patch.split('/')[-1][:-4]
    if os.path.exists(os.path.join(out_dir, fname+".pt")):
        continue

    image_path = os.path.join(root_dir, image_patch)
    ela_path = os.path.join(root_dir, ela)

    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (256,256), cv2.INTER_AREA)

    ela_image = cv2.imread(ela_path, cv2.IMREAD_COLOR)
    ela_image = cv2.cvtColor(ela_image, cv2.COLOR_BGR2RGB)
    ela_image = cv2.resize(ela_image, (256,256), cv2.INTER_AREA)
    
    tensor = ensemble(model, image, ela_image).cpu().detach()
    # print(type(tensor))
    # print(tensor.shape)
    torch.save(tensor, os.path.join(out_dir, fname+".pt"))

In [10]:
x = torch.load('Image_Manipulation_Dataset/CASIA_2.0/1792_tensors/Tp_D_CRN_M_N_pla10110_sec00046_10119.pt')
type(x)

torch.Tensor

In [14]:
rows = []

In [15]:
files = os.listdir('Image_Manipulation_Dataset/CASIA_2.0/1792_tensors')

for x in files:
    if x.startswith('Au'):
        rows.append({
            'fname':x,
            'label':0
        })
    elif x.startswith('Tp'):
        rows.append({
            'fname':x,
            'label':1
        })

In [20]:
df2 = pd.DataFrame(rows)
df2["fold"] = -1  
y = df2.label.values
kf = model_selection.StratifiedKFold(n_splits=10)

for f, (t_, v_) in enumerate(kf.split(X=df2, y=y)):
    df2.loc[v_, 'fold'] = f

df2

,fname,label,fold
0,Au_ani_00001.pt,0,0
1,Au_ani_00003.pt,0,0
2,Au_ani_00004.pt,0,0
3,Au_ani_00005.pt,0,0
4,Au_ani_00007.pt,0,0
...,...,...,...
5995,Au_cha_30375.pt,0,9
5996,Au_cha_30376.pt,0,9
5997,Au_cha_30377.pt,0,9
5998,Au_cha_30380.pt,0,9


In [23]:
df2.to_csv('1792_tensors.csv',index=False)